In [2]:
import pickle
import numpy as np
import pandas as pd  
import matplotlib.pyplot as plt  
import seaborn as sns  
import scipy.stats as ss  
import pingouin as pg
from statsmodels.stats.multicomp import MultiComparison
import statsmodels.api as sm
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import statsmodels.formula.api as smf 
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from xgboost import XGBRegressor, plot_importance
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from statsmodels.tsa.statespace.sarimax import SARIMAX
from scipy.stats import randint, uniform
from sklearn.tree import DecisionTreeRegressor, plot_tree
from sklearn.linear_model import ridge_regression, Lasso, Ridge
import random
import locale
from tabulate import tabulate
from sklearn.model_selection import GridSearchCV



# Inter
from statsmodels.formula.api import ols
import shap
import lime 
import lime.lime_tabular

# SARIMAX
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.tsa.stattools import adfuller
from statsmodels.graphics.tsaplots import plot_pacf


c:\Users\sarah\miniconda3\envs\raku\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [5]:
# Chargement des modèles préalablement entraînés
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/model_lr.pkl', 'rb') as file:
    model_lr = pickle.load(file)
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/model_rf.pkl', 'rb') as file:
    model_rf = pickle.load(file)
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/best_xgb_model.pkl', 'rb') as file:
    best_xgb_model = pickle.load(file)
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/model_ridge.pkl', 'rb') as file:
    model_ridge = pickle.load(file)
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/model_lasso.pkl', 'rb') as file:
    model_lasso = pickle.load(file)
with open('C:/Users/sarah/OneDrive/Documents/GitHub/DS_RakutenVF/00 - Annexes/models/model_dt.pkl', 'rb') as file:
    model_dt = pickle.load(file)


c:\Users\sarah\miniconda3\envs\raku\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator DecisionTreeRegressor from version 1.4.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sarah\miniconda3\envs\raku\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator RandomForestRegressor from version 1.4.0 when using version 1.4.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(
c:\Users\sarah\miniconda3\envs\raku\lib\site-packages\sklearn\base.py:376: InconsistentVersionWarning: Trying to unpickle estimator Ridge from version 1.4.0 when using vers

## 1. Régression linéaire multiple interpretabilité

### Global

In [6]:
model_lr_int = LinearRegression()
model_lr_int.fit(X_train, y_train)

coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model_lr_int.coef_
})
print("Coefficients of Linear Regression Model:")
print(coefficients)

NameError: name 'LinearRegression' is not defined

In [ ]:
explainer_lr = shap.Explainer(model_lr_int, X_test)
shap_values_lr = explainer_lr(X_test)
shap.initjs()
shap.summary_plot(shap_values_lr, X_test)

## 2. random forest interpretabilité

In [ ]:
importances_rf = model_rf.feature_importances_
indices_rf = np.argsort(importances_rf)[::-1]

print("Feature importances for Random Forest:")
for f in range(X.shape[1]):
    print(f"{X.columns[indices_rf[f]]}: {importances_rf[indices_rf[f]]}")

In [ ]:
explainer_lime = lime.lime_tabular.LimeTabularExplainer(X_train.values, feature_names= X_train.columns, class_names= ['Prix'], mode='regression')

In [ ]:
i = 0
exp_rf = explainer_lime.explain_instance(X_test.values[i], model_rf.predict)
exp_rf.show_in_notebook(show_table=True)

## 3. XGBOOOST interpretabilité

In [ ]:
plot_importance(best_xgb_model)
plt.show()

In [ ]:
explainer_xgb = shap.Explainer(best_xgb_model, X_test)
shap_values_xgb = explainer_xgb(X_test)
shap.initjs()
shap.summary_plot(shap_values_xgb, X_test)

## 4. Régression Ridge et Lasso intérprétabilité

###  Global

In [ ]:
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model_ridge.coef_
})
print("Coefficients of ridge Model:")
print(coefficients)

In [ ]:
coefficients = pd.DataFrame({
    'Feature': X.columns,
    'Coefficient': model_lasso.coef_
})
print("Coefficients of lasso Model:")
print(coefficients)

### Local

In [ ]:
explainer_ridge = shap.Explainer(model_ridge, X_test)
shap_values_ridge = explainer_ridge(X_test)
shap.initjs()
shap.summary_plot(shap_values_ridge, X_test)

In [ ]:
explainer_lasso = shap.Explainer(model_lasso, X_test)
shap_values_lasso = explainer_lasso(X_test)
shap.initjs()
shap.summary_plot(shap_values_lasso, X_test)

## 5. Modèles d'arbres de décision interprétabilité


In [ ]:
importances_dt = model_dt.feature_importances_
indices_dt = np.argsort(importances_dt)[::-1]

print("Feature importances for decision tree:")
for f in range(X.shape[1]):
    print(f"{X.columns[indices_dt[f]]}: {importances_dt[indices_dt[f]]}")

### Local

In [ ]:
i = 9
exp_dt = explainer_lime.explain_instance(X_test.values[i], model_dt.predict)
exp_dt.show_in_notebook(show_table=True)